In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
import re
import torch
import tqdm
import pandas as pd
import tqdm 
from transformers import BertModel, BertTokenizer
import pandas as pd
import pickle as pkl
import datasets 
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
btok = AutoTokenizer.from_pretrained('bert-base-uncased')
bmod = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained("wukevin/tcr-bert", do_lower_case=False)
model = BertModel.from_pretrained("wukevin/tcr-bert")

# if torch.cuda.is_available():
model.to('cuda')
model = model.half()
    
def BERT_embedding(x):
    seq = " ".join(x)
    seq = re.sub(r"[UZOB]", "X", seq)
    encoded_input = tokenizer(seq, return_tensors='pt')
    output = model(**encoded_input)
    return output

/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-13 14:26:01.168712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734125161.180386     382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734125161.183978     382 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 14:26:01.196884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-crit

In [3]:
dfetr=pd.read_csv('./data/BAP/epi_split/train.csv', header=None)
dfete=pd.read_csv('./data/BAP/epi_split/test.csv', header=None)
dfetr.columns = ['Epitope', 'TCR', 'Label']
dfete.columns = ['Epitope', 'TCR', 'Label']

dfttr=pd.read_csv('./data/BAP/tcr_split/train.csv', header=None)
dftte=pd.read_csv('./data/BAP/tcr_split/test.csv', header=None)
dfttr.columns = ['Epitope', 'TCR', 'Label']
dftte.columns = ['Epitope', 'TCR', 'Label']


dfho_epi=pd.read_csv('./data/true_hold_out/epitope_split_test.csv', header=None)
dfho_tcr=pd.read_csv('./data/true_hold_out/tcr_split_test.csv', header=None)
dfho_epi.columns = ['Epitope', 'TCR'] 
dfho_tcr.columns = ['Epitope', 'TCR']

dfemb=pd.read_csv('./data/embedding/TCRrepertoires.csv', header=None)


In [5]:
def proc_input_dct(inpdct):
    updated_dct = {}
    for key, value in inpdct.items():
        if len(value.shape)==3:
            updated_dct[key] = value.mean(axis=1)
        else:
            updated_dct[key] = value
    return updated_dct



# Generate the Classification Results for T5

In [47]:
#---------------------
# dct = pkl.load(open('./embT5/epi_train_dct.pkl', 'rb'))
# dct = proc_input_dct(dct)
# dctte = pkl.load(open('./embT5/epi_test_dct.pkl', 'rb'))
# dctte = proc_input_dct(dctte)

# y_train = dfetr.Label.values
# y_test = dfete.Label.values

#---------------------
dct = pkl.load(open('./embT5/tcr_train_dct.pkl', 'rb'))
dct = proc_input_dct(dct)
dctte = pkl.load(open('./embT5/tcr_test_dct.pkl', 'rb'))
dctte = proc_input_dct(dctte)

y_train = dfttr.Label.values
y_test = dftte.Label.values

#---------------------
X_train = np.concatenate((dct['epi_full'], dct['tcr_full']), axis=1) 
X_test = np.concatenate((dctte['epi_full'], dctte['tcr_full']), axis=1)
# dctte = pkl.load(open('./embT5/epi_test_dct.pkl', 'rb'))

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score 
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

# Split data into train and test
# Normalize -> PCA -> SVR Pipeline
svr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
#     ('svr', SVC(keprobability=='rbf', class_weight='balanced', probability=True))
    ('svr', SVR())
])

# Fit and predict
svr_pipeline.fit(X_train, y_train)
y_pred_svr = svr_pipeline.predict(X_test)

# Since SVR is a regressor, convert predictions to binary labels for classification metrics
y_pred_svr_binary = (y_pred_svr > 0.5).astype(int)

# Metrics for SVR
accuracy_svr = accuracy_score(y_test, y_pred_svr_binary)
rocauc_svr = roc_auc_score(y_test, y_pred_svr_binary)
f1_svr = f1_score(y_test, y_pred_svr_binary)
prec_score = precision_score(y_test, y_pred_svr_binary)
rec_score = recall_score(y_test, y_pred_svr_binary)

print("SVR Metrics:")
print(f"Accuracy: {accuracy_svr:.4f}")
print(f"ROC AUC: {rocauc_svr:.4f}")
print(f"F1 Score: {f1_svr:.4f}")
print(f"Precision Score: {prec_score:.4f}")
print(f"Recall Score: {rec_score:.4f}")

/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:493: Futur

SVR Metrics:
Accuracy: 0.7306
ROC AUC: 0.7304
F1 Score: 0.7054
Precision Score: 0.7761
Recall Score: 0.6465


In [48]:
#---------------------
dct = pkl.load(open('./embT5/epi_train_dct.pkl', 'rb'))
dct = proc_input_dct(dct)
dctte = pkl.load(open('./embT5/epi_test_dct.pkl', 'rb'))
dctte = proc_input_dct(dctte)

y_train = dfetr.Label.values
y_test = dfete.Label.values

#---------------------
# dct = pkl.load(open('./embT5/tcr_train_dct.pkl', 'rb'))
# dct = proc_input_dct(dct)
# dctte = pkl.load(open('./embT5/tcr_test_dct.pkl', 'rb'))
# dctte = proc_input_dct(dctte)

# y_train = dfttr.Label.values
# y_test = dftte.Label.values

#---------------------
X_train = np.concatenate((dct['epi_full'], dct['tcr_full']), axis=1) 
X_test = np.concatenate((dctte['epi_full'], dctte['tcr_full']), axis=1)
# dctte = pkl.load(open('./embT5/epi_test_dct.pkl', 'rb'))

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

# Split data into train and test
# Normalize -> PCA -> SVR Pipeline
svr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
#     ('svr', SVC(kernel='rbf', class_weight='balanced', probability=True))
    ('svr', SVC())
])

# Fit and predict
svr_pipeline.fit(X_train, y_train)
y_pred_svr = svr_pipeline.predict(X_test)

# Since SVR is a regressor, convert predictions to binary labels for classification metrics
y_pred_svr_binary = (y_pred_svr > 0.5).astype(int)

# Metrics for SVR
accuracy_svr = accuracy_score(y_test, y_pred_svr_binary)
rocauc_svr = roc_auc_score(y_test, y_pred_svr_binary)
f1_svr = f1_score(y_test, y_pred_svr_binary)
prec_score = precision_score(y_test, y_pred_svr_binary)
rec_score = recall_score(y_test, y_pred_svr_binary)

print("SVR Metrics:")
print(f"Accuracy: {accuracy_svr:.4f}")
print(f"ROC AUC: {rocauc_svr:.4f}")
print(f"F1 Score: {f1_svr:.4f}")
print(f"Precision Score: {prec_score:.4f}")
print(f"Recall Score: {rec_score:.4f}")

/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/prad/anaconda3/envs/tcrbert/lib/python3.9/site-packages/sklearn/base.py:493: Futur

SVR Metrics:
Accuracy: 0.6767
ROC AUC: 0.6767
F1 Score: 0.6406
Precision Score: 0.7212
Recall Score: 0.5762
